In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
import h5py

import plotly.graph_objects as go

In [2]:
%matplotlib inline

In [3]:
from matplotlib import rcParams
rcParams['figure.figsize'] = [16, 8]
rcParams['font.family'] = ['sans-serif']
rcParams['axes.unicode_minus'] = False 

In [4]:
from matplotlib.font_manager import _rebuild
_rebuild()

In [ ]:
f1 = h5py.File('../data/000007.h5','r')
f2 = h5py.File('/Users/d/code/python/xx/000007.h5','r')

In [ ]:
for i in f1['/stock'].items():
    print(i)

In [ ]:
df1 = pd.read_hdf('../data/000007.h5', '/stock/kline5')
df1_1 = pd.read_hdf('../data/000007-1.h5', '/stock/kline5')
df2 = pd.read_hdf('/Users/d/code/python/xx/000007.h5', '/stock/kline5')

In [ ]:
df1.tail()

In [ ]:
df1_1[-50:]

In [ ]:
df2[-50:]

In [ ]:
del df['date']
del df['volume']

In [ ]:
p = df.plot()

In [ ]:
# p.savefig('../data/000007', dpi=300)
fig = p.get_figure()
fig.savefig("../data/000007-1.png", dpi=300)

In [ ]:
pd.read_hdf('../data/002162.h5', '/stock/details')

In [ ]:
stocks = pd.read_hdf('../data/002162.h5', '/stock/stocks')

In [ ]:
stocks

In [ ]:
f = h5py.File('../data/002162.h5','r')

In [ ]:
for i in f.items():
    print(i)

In [ ]:
for i in f['/stock'].items():
    print(i)

In [ ]:
training_set = stocks['closing_price'].values
training_set = training_set.reshape(training_set.shape[0], 1)

In [ ]:
plt.plot(training_set, label = '实际数据')
plt.legend()

In [ ]:
def sliding_windows(data, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x), np.array(y)

sc = MinMaxScaler()
training_data = sc.fit_transform(training_set)

seq_length = 20
x, y = sliding_windows(training_data, seq_length)

train_size = int(len(y) * 0.50)
test_size = len(y) - train_size

dataX = Variable(torch.Tensor(np.array(x)))
dataY = Variable(torch.Tensor(np.array(y)))

trainX = Variable(torch.Tensor(np.array(x[:train_size])))
trainY = Variable(torch.Tensor(np.array(y[:train_size])))

testX = Variable(torch.Tensor(np.array(x[train_size:])))
testY = Variable(torch.Tensor(np.array(y[train_size:])))

In [ ]:
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.fc(h_out)
        
        return out

In [ ]:
class GRU(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(GRU, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        
        self.gru = nn.GRU(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
#         c_0 = Variable(torch.zeros(
#             self.num_layers, x.size(0), self.hidden_size))
        
        # Propagate input through LSTM
        ula, h_out = self.gru(x, (h_0))
        
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.fc(h_out)
        
        return out

In [ ]:

num_epochs = 3000
learning_rate = 0.01

input_size = 1
hidden_size = 2
num_layers = 1

num_classes = 1

# lstm = LSTM(num_classes, input_size, hidden_size, num_layers)
lstm = GRU(num_classes, input_size, hidden_size, num_layers)

criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    outputs = lstm(trainX)
    optimizer.zero_grad()
    
    # obtain the loss function
    loss = criterion(outputs, trainY)
    
    loss.backward()
    
    optimizer.step()
    if epoch % 100 == 0:
      print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

In [ ]:
lstm.eval()

In [ ]:
pres = []
pre = None
for i in range(testX.shape[0]):
    if pre is None:
        pre = lstm(testX[i].reshape(1, seq_length, 1))
    else:
        d = testX[i]
        d[-1] = pre.item()
        pre = lstm(d.reshape(1, seq_length, 1))
    pres.append(pre.item())
    
dp = sc.inverse_transform(np.asarray(pres).reshape(testX.shape[0], 1))
xpd = np.asarray(range(train_size, train_size + test_size))

In [ ]:
train_predict = lstm(dataX)
data_predict = train_predict.data.numpy()
dataY_plot = dataY.data.numpy()

data_predict = sc.inverse_transform(data_predict)
dataY_plot = sc.inverse_transform(dataY_plot)

plt.axvline(x=train_size, c='r', linestyle='--')

plt.plot(dataY_plot, label='真实数据')
plt.plot(data_predict, label='预测数据', alpha=0.7)
# plt.plot(xpd, dp, label='二次预测')
plt.suptitle('随机数序列预测')
plt.legend()
plt.savefig('../data/002162', dpi=300)
plt.show()

In [17]:
st = h5py.File('../data/sh000062.h5','r')

In [18]:
for i in st['/stock'].items():
    print(i)

('details', <HDF5 dataset "details": shape (0,), type "|V62">)
('kline15', <HDF5 dataset "kline15": shape (1023,), type "|V28">)
('kline30', <HDF5 dataset "kline30": shape (752,), type "|V28">)
('kline5', <HDF5 dataset "kline5": shape (1023,), type "|V28">)
('kline60', <HDF5 dataset "kline60": shape (472,), type "|V28">)
('stocks', <HDF5 dataset "stocks": shape (0,), type "|V40">)


In [19]:
df = pd.read_hdf('../data/sh000062.h5', '/stock/kline5')

In [20]:
df.head()

,date,open,high,close,low,volume
0,1595569800,2061.298096,2061.780029,2052.461914,2052.455078,52126300.0
1,1595570100,2052.141113,2057.027100,2055.866943,2051.787109,38288700.0
2,1595570400,2055.866943,2055.993896,2049.029053,2048.000977,51116700.0
3,1595570700,2049.025879,2049.025879,2043.984985,2042.780029,65422500.0
4,1595571000,2043.369995,2043.418945,2033.948975,2031.973999,75130800.0


In [21]:
date = df['date']
open = df['open']
high = df['high']
close = df['close']
low = df['low']
volume = df['volume']

In [22]:
date[0]

1595569800

In [ ]:
print(pd.to_datetime(1594878000.0, unit='s'))
print(pd.Timestamp.fromtimestamp(1594878000.0))
print(pd.Timestamp.fromtimestamp(1.594878e+09))
print(pd.Timestamp.fromtimestamp(1594877952.0))

In [ ]:
print(date[0])

# 1.594878e+09
pd.to_datetime('1594878000.0', unit='s')

In [ ]:
pd.Timestamp.fromtimestamp('')

In [ ]:
date.astype(np.str)

In [ ]:
#date.astype(np.float).apply(lambda x: print(x))

In [23]:
date.apply(lambda x: pd.Timestamp.fromtimestamp(float(x)))

0      2020-07-24 13:50:00
1      2020-07-24 13:55:00
2      2020-07-24 14:00:00
3      2020-07-24 14:05:00
4      2020-07-24 14:10:00
               ...        
1018   2020-08-24 14:40:00
1019   2020-08-24 14:45:00
1020   2020-08-24 14:50:00
1021   2020-08-24 14:55:00
1022   2020-08-24 15:00:00
Name: date, Length: 1023, dtype: datetime64[ns]

In [ ]:
# pd.to_datetime(df['date'])
print(date[0])
pd.to_datetime(df['date'], unit='s')

In [ ]:
date.plot()

In [ ]:
df.plot()

In [6]:
df = pd.read_hdf('/Users/d/code/python/xx/stock/sz002319.h5', '/stock/kline5')

In [7]:
df

,date,open,high,close,low,volume
0,1594263300,6.57,6.58,6.57,6.57,18600.0
1,1594263600,6.56,6.56,6.56,6.56,6100.0
2,1594263900,6.56,6.57,6.57,6.56,19400.0
3,1594264200,6.57,6.60,6.58,6.57,23100.0
4,1594264500,6.60,6.61,6.60,6.58,26400.0
...,...,...,...,...,...,...
1017,1598250900,7.63,7.66,7.64,7.63,40500.0
1018,1598251200,7.64,7.67,7.67,7.64,22200.0
1019,1598251500,7.67,7.77,7.74,7.66,97100.0
1020,1598251800,7.73,7.73,7.72,7.70,15300.0


In [8]:
df.date.apply(lambda x: pd.Timestamp.fromtimestamp(x))

0      2020-07-09 10:55:00
1      2020-07-09 11:00:00
2      2020-07-09 11:05:00
3      2020-07-09 11:10:00
4      2020-07-09 11:15:00
               ...        
1017   2020-08-24 14:35:00
1018   2020-08-24 14:40:00
1019   2020-08-24 14:45:00
1020   2020-08-24 14:50:00
1021   2020-08-24 14:55:00
Name: date, Length: 1022, dtype: datetime64[ns]

In [5]:
df = pd.read_hdf('/Users/d/code/python/left5/data/sh600236.h5', '/stock/kline5')

In [6]:
df

,date,open,high,close,low,volume
0,1595569800,4.36,4.36,4.35,4.34,366960.0
1,1595570100,4.35,4.35,4.35,4.34,71773.0
2,1595570400,4.35,4.35,4.34,4.33,142627.0
3,1595570700,4.34,4.35,4.32,4.32,362790.0
4,1595571000,4.32,4.33,4.32,4.31,151400.0
...,...,...,...,...,...,...
1080,1598408400,4.80,4.81,4.81,4.80,61320.0
1081,1598408700,4.80,4.81,4.81,4.80,36500.0
1082,1598409000,4.81,4.81,4.81,4.80,84940.0
1083,1598409300,4.80,4.81,4.81,4.80,54600.0


In [7]:
df.date.apply(lambda x: pd.Timestamp.fromtimestamp(x))

0      2020-07-24 13:50:00
1      2020-07-24 13:55:00
2      2020-07-24 14:00:00
3      2020-07-24 14:05:00
4      2020-07-24 14:10:00
               ...        
1080   2020-08-26 10:20:00
1081   2020-08-26 10:25:00
1082   2020-08-26 10:30:00
1083   2020-08-26 10:35:00
1084   2020-08-26 10:40:00
Name: date, Length: 1085, dtype: datetime64[ns]

In [12]:
training_set = df.to_numpy()[:, 1:]

In [13]:
sc = MinMaxScaler()
training_data = sc.fit_transform(training_set)

In [16]:
training_data

array([[0.07017541, 0.05000036, 0.05263114, 0.05660415, 0.07856112],
       [0.05263114, 0.03333331, 0.05263114, 0.05660415, 0.01498446],
       [0.05263114, 0.03333331, 0.0350877 , 0.0377358 , 0.03024483],
       ...,
       [0.85964918, 0.80000016, 0.85964918, 0.9245284 , 0.01782034],
       [0.84210575, 0.80000016, 0.85964918, 0.9245284 , 0.01128578],
       [0.84210575, 0.80000016, 0.84210575, 0.9245284 , 0.03741238]])

In [27]:
sc = MinMaxScaler()
sc.fit_transform(df[['open', 'high', 'close', 'low']].values)

array([[0.07017517, 0.05000019, 0.0526309 , 0.05660343],
       [0.0526309 , 0.0333333 , 0.0526309 , 0.05660343],
       [0.0526309 , 0.0333333 , 0.03508759, 0.03773594],
       ...,
       [0.8596487 , 0.8000002 , 0.8596487 , 0.9245281 ],
       [0.84210587, 0.8000002 , 0.8596487 , 0.9245281 ],
       [0.84210587, 0.8000002 , 0.84210587, 0.9245281 ]], dtype=float32)

In [41]:
def sliding_windows(data, seq_length, pre_size):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length:i+seq_length+pre_size]
        x.append(_x)
        y.append(_y)

    return np.array(x), np.array(y)

sc = MinMaxScaler()
training_data = sc.fit_transform(df[['open', 'high', 'close', 'low']].values)

seq_length = 4
x, y = sliding_windows(training_data, seq_length, 1)

train_size = int(len(y) * 0.67)
test_size = len(y) - train_size

dataX = Variable(torch.Tensor(np.array(x)))
dataY = Variable(torch.Tensor(np.array(y)))

trainX = Variable(torch.Tensor(np.array(x[:train_size])))
trainY = Variable(torch.Tensor(np.array(y[:train_size])))

testX = Variable(torch.Tensor(np.array(x[train_size:])))
testY = Variable(torch.Tensor(np.array(y[train_size:])))

In [42]:
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.fc(h_out)
        
        return out

In [43]:
num_epochs = 3000
learning_rate = 0.01

input_size = 4
hidden_size = 2
num_layers = 1

num_classes = 4

lstm = LSTM(num_classes, input_size, hidden_size, num_layers)

criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    outputs = lstm(trainX)
    optimizer.zero_grad()
    
    # obtain the loss function
    loss = criterion(outputs, trainY)
    
    loss.backward()
    
    optimizer.step()
    if epoch % 100 == 0:
      print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

/Users/d/venv/lib/python3.7/site-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([723, 1, 4])) that is different to the input size (torch.Size([723, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 0, loss: 0.42881
Epoch: 100, loss: 0.00445
Epoch: 200, loss: 0.00434
Epoch: 300, loss: 0.00431
Epoch: 400, loss: 0.00430
Epoch: 500, loss: 0.00429
Epoch: 600, loss: 0.00429
Epoch: 700, loss: 0.00429
Epoch: 800, loss: 0.00429
Epoch: 900, loss: 0.00429
Epoch: 1000, loss: 0.00429
Epoch: 1100, loss: 0.00429
Epoch: 1200, loss: 0.00429
Epoch: 1300, loss: 0.00429
Epoch: 1400, loss: 0.00429
Epoch: 1500, loss: 0.00429
Epoch: 1600, loss: 0.00429
Epoch: 1700, loss: 0.00429
Epoch: 1800, loss: 0.00429
Epoch: 1900, loss: 0.00429
Epoch: 2000, loss: 0.00429
Epoch: 2100, loss: 0.00429
Epoch: 2200, loss: 0.00429
Epoch: 2300, loss: 0.00429
Epoch: 2400, loss: 0.00429
Epoch: 2500, loss: 0.00429
Epoch: 2600, loss: 0.00429
Epoch: 2700, loss: 0.00429
Epoch: 2800, loss: 0.00429
Epoch: 2900, loss: 0.00429


In [ ]:
lstm.eval()
train_predict = lstm(dataX)

data_predict = train_predict.data.numpy()
dataY_plot = dataY.data.numpy()

data_predict = sc.inverse_transform(data_predict)
dataY_plot = sc.inverse_transform(dataY_plot.reshape(-1, 4))

plt.axvline(x=train_size, c='r', linestyle='--')

plt.plot(dataY_plot[:,2], label=u'真实数据')
plt.plot(data_predict[:,2], label=u'预测数据')
plt.suptitle('随机数序列预测')
plt.legend()
plt.show()

In [8]:
dd = df

In [9]:
dd.date = df.date.apply(lambda x: pd.Timestamp.fromtimestamp(x))

In [10]:
dd

,date,open,high,close,low,volume
0,2020-07-24 13:50:00,4.36,4.36,4.35,4.34,366960.0
1,2020-07-24 13:55:00,4.35,4.35,4.35,4.34,71773.0
2,2020-07-24 14:00:00,4.35,4.35,4.34,4.33,142627.0
3,2020-07-24 14:05:00,4.34,4.35,4.32,4.32,362790.0
4,2020-07-24 14:10:00,4.32,4.33,4.32,4.31,151400.0
...,...,...,...,...,...,...
1080,2020-08-26 10:20:00,4.80,4.81,4.81,4.80,61320.0
1081,2020-08-26 10:25:00,4.80,4.81,4.81,4.80,36500.0
1082,2020-08-26 10:30:00,4.81,4.81,4.81,4.80,84940.0
1083,2020-08-26 10:35:00,4.80,4.81,4.81,4.80,54600.0


In [11]:
go.Candlestick?

In [12]:
INCREASING_COLOR = '#17BECF'
DECREASING_COLOR = '#7F7F7F'

In [13]:
dd['ma5'] = df.close.rolling(window=5, min_periods=1).mean()
dd['ma10'] = df.close.rolling(window=10, min_periods=1).mean()

In [14]:
from lutils.stock import plot_stock

In [15]:
fig = plot_stock(dd)

In [17]:
import chart_studio.plotly as py

In [ ]:
py.iplot(fig)

In [19]:
def bbands(price, window_size=10, num_of_std=5):
    rolling_mean = price.rolling(window=window_size).mean()
    rolling_std  = price.rolling(window=window_size).std()
    upper_band = rolling_mean + (rolling_std*num_of_std)
    lower_band = rolling_mean - (rolling_std*num_of_std)
    return rolling_mean, upper_band, lower_band

bb_avg, bb_upper, bb_lower = bbands(dd.close)

In [27]:
fig = go.Figure(data=[
    go.Candlestick(
        #x=dd.date,
        open=dd.open,
        high=dd.high,
        low=dd.low,
        close=dd.close,
        name='000',
        yaxis='y2'),
    go.Scatter(y=df.ma5, line=dict(color='orange', width=1), name='MA 5', yaxis='y2'),
    go.Scatter(y=df.ma10, line=dict(color='green', width=1), name='MA 10', yaxis='y2'),
    
#     go.Scatter(y=bb_avg, line=dict(color='#ccc', width=1), name='MA 10'),
    go.Scatter(y=bb_upper, line=dict(color='#ccc', width=1), legendgroup='Bollinger Bands', name='Bollinger Bands', yaxis='y2'),
    go.Scatter(y=bb_lower, line=dict(color='#ccc', width=1), legendgroup='Bollinger Bands', showlegend=False, yaxis='y2'),
    
    go.Bar(y=dd.volume, yaxis='y', name='volume')
])

fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
# y=df.Volume, 
# marker=dict( color=colors ),
# type='bar', yaxis='y', name='Volume'

fig.show()